### This file is for testing radical changes to the codebase. Upside Down Reinforcement Learning.py is the working version.

In [166]:
import gym
env = gym.make('LunarLander-v2')

In [167]:
#command takes form [derired reward, desired horizon]
import numpy as np
def random_policy(obs, command):
    return np.random.randint(env.action_space.n)

In [168]:
import time
from copy import deepcopy
#Visualise agent function
def visualise_agent(policy, command, n=5):
    try:
        for trial_i in range(n):
            current_command = deepcopy(command)
            observation = env.reset()
            done=False
            t=0
            episode_return=0
            while not done:
                env.render()
                action = policy(torch.tensor([observation]).double(), torch.tensor([command]).double())
                observation, reward, done, info = env.step(action)
                episode_return+=reward
                current_command[0]-= reward
                current_command[1] = max(1, current_command[1]-1)
                t+=1
            env.render()
            time.sleep(1.5)
            print("Episode {} finished after {} timesteps. Return = {}".format(trial_i, t, episode_return))
        env.close()
    except KeyboardInterrupt:
        env.close()

In [169]:
#visualise_agent(random_policy, command=[500, 500], 1)

In [170]:
import torch
import torch.nn.functional as F

class FCNN_AGENT(torch.nn.Module):
    def __init__(self, command_scale):
        super().__init__()
        hidden_size=64
        self.command_scale=command_scale
        self.observation_embedding = torch.nn.Sequential(
            torch.nn.Linear(np.prod(env.observation_space.shape), hidden_size),
            torch.nn.Sigmoid()
        )
        self.command_embedding = torch.nn.Sequential(
            torch.nn.Linear(2, hidden_size),
            torch.nn.Sigmoid()
        )
        self.to_output = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, env.action_space.n)
        )
    
    def forward(self, observation, command):
        obs_emebdding = self.observation_embedding(observation)
        cmd_embedding = self.command_embedding(command*self.command_scale)
        embedding = torch.mul(obs_emebdding, cmd_embedding)
        action_prob_logits = self.to_output(embedding)
        return action_prob_logits
    
    def create_optimizer(self, lr):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

In [171]:
from copy import deepcopy
def collect_experience(policy, replay_buffer, replay_size, last_few, n_episodes=100, log_to_tensorboard=True):
    global i_episode
    init_replay_buffer = deepcopy(replay_buffer)
    try:
        for _ in range(n_episodes):
            command = sample_command(init_replay_buffer, last_few)
            writer.add_scalar('Command desired reward/Episode', command[0], i_episode)    # write loss to a graph
            writer.add_scalar('Command horizon/Episode', command[1], i_episode)    # write loss to a graph
            observation = env.reset()
            episode_mem = {'observation':[],
                           'action':[],
                           'reward':[],}
            done=False
            while not done:
                action = policy(torch.tensor([observation]).double(), torch.tensor([command]).double())
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                
                observation=new_observation
                command[0]-= reward
                command[1] = max(1, command[1]-1)
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            replay_buffer.append(episode_mem)
            i_episode+=1
            if log_to_tensorboard: writer.add_scalar('Return/Episode', sum(episode_mem['reward']), i_episode)    # write loss to a graph
            print("Episode {} finished after {} timesteps. Return = {}".format(i_episode, len(episode_mem['observation']), sum(episode_mem['reward'])))
        env.close()
    except KeyboardInterrupt:
        env.close()
    replay_buffer = sorted(replay_buffer, key=lambda x:x['return'])[-replay_size:]
    return replay_buffer

def sample_command(replay_buffer, last_few):
    if len(replay_buffer)==0:
        return [1, 1]
    else:
        command_samples = replay_buffer[-last_few:]
        lengths = [mem['episode_len'] for mem in command_samples]
        returns = [mem['return'] for mem in command_samples]
        mean_return, std_return = np.mean(returns), np.std(returns)
        command_horizon = np.mean(lengths)
        desired_reward = np.random.uniform(mean_return, mean_return+std_return)
        return [desired_reward, command_horizon]

In [172]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64, log_to_tensorboard=True):
    global i_updates
    all_costs = []
    for i in range(n_updates):
        batch_observations = np.zeros((batch_size, np.prod(env.observation_space.shape)))
        batch_commands = np.zeros((batch_size, 2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_t1 = np.random.randint(0, len(replay_buffer[sample_episode]['observation']))
            sample_t2 = len(replay_buffer[sample_episode]['observation'])
            ##sample_t2 = np.random.randint(sample_t1+1, len(replay_buffer[sample_episode]['observation'])+1)
            sample_horizon = sample_t2-sample_t1
            sample_mem = replay_buffer[sample_episode]['observation'][sample_t1]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_t1:sample_t2])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_t1]
            batch_observations[b] = sample_mem
            batch_commands[b] = [sample_desired_reward, sample_horizon]
            batch_label[b] = label
        batch_observations = torch.tensor(batch_observations).double()
        batch_commands = torch.tensor(batch_commands).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_observations, batch_commands)
        cost = F.cross_entropy(pred, batch_label)
        if log_to_tensorboard: writer.add_scalar('Cost/NN update', cost.item() , i_updates)    # write loss to a graph
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
        i_updates+=1
    return np.mean(all_costs)

In [173]:
def create_greedy_policy(policy_network):
    def policy(obs, command):
        action_logits = policy_network(obs, command)
        action_probs = F.softmax(action_logits, dim=-1)
        action = np.argmax(action_probs.detach().numpy())
        return action
    return policy

def create_stochastic_policy(policy_network):
    def policy(obs, command):
        action_logits = policy_network(obs, command)
        action_probs = F.softmax(action_logits, dim=-1)
        action = torch.distributions.Categorical(action_probs).sample().item()
        return action
    return policy

In [175]:
i_episode=0
i_updates=0 #number of parameter updates to the neural network
replay_buffer = []
log_to_tensorboard = True 

replay_size = 700
last_few = 50
batch_size = 256
n_warm_up_episodes = 50
n_episodes_per_iter = 25
n_updates_per_iter = 15
command_scale = 0.02
lr = 0.001

agent = FCNN_AGENT(command_scale).double()
agent.create_optimizer(lr)

stochastic_policy = create_stochastic_policy(agent)
greedy_policy = create_greedy_policy(agent)

In [176]:
# SET UP TRAINING VISUALISATION
# SET UP TRAINING VISUALISATION
if log_to_tensorboard: from torch.utils.tensorboard import SummaryWriter
if log_to_tensorboard: writer = SummaryWriter() # we will use this to show our models performance on a graph using tensorboard

In [177]:
#Collect warm up episodes
replay_buffer = collect_experience(random_policy, replay_buffer, replay_size, last_few, n_warm_up_episodes, log_to_tensorboard)
train_net(agent, replay_buffer, n_updates_per_iter, batch_size, log_to_tensorboard)

Episode 1 finished after 64 timesteps. Return = -84.60078681057627
Episode 2 finished after 95 timesteps. Return = -120.78480124507686
Episode 3 finished after 63 timesteps. Return = -172.14825381248892
Episode 4 finished after 71 timesteps. Return = -91.4286045161926
Episode 5 finished after 88 timesteps. Return = -94.85860439292763
Episode 6 finished after 103 timesteps. Return = -243.6307219272219
Episode 7 finished after 94 timesteps. Return = -321.5253342006809
Episode 8 finished after 123 timesteps. Return = -198.62381016401844
Episode 9 finished after 71 timesteps. Return = -132.11189604460625
Episode 10 finished after 67 timesteps. Return = -124.86243874045171
Episode 11 finished after 85 timesteps. Return = -133.11606950032888
Episode 12 finished after 67 timesteps. Return = -71.45064283024958
Episode 13 finished after 92 timesteps. Return = -253.01980387275094
Episode 14 finished after 127 timesteps. Return = -106.47646839864615
Episode 15 finished after 70 timesteps. Return 

1.385637102955703

In [178]:
n_iters = 1000
for i in range(n_iters):
    replay_buffer = collect_experience(stochastic_policy, replay_buffer, replay_size, last_few, n_episodes_per_iter, log_to_tensorboard)
    train_net(agent, replay_buffer, n_updates_per_iter, batch_size, log_to_tensorboard)

Episode 51 finished after 107 timesteps. Return = -113.07739390702349
Episode 52 finished after 80 timesteps. Return = -316.08569869815676
Episode 53 finished after 66 timesteps. Return = -36.60251067892713
Episode 54 finished after 87 timesteps. Return = -69.98392383362653
Episode 55 finished after 101 timesteps. Return = -270.38410966312466
Episode 56 finished after 111 timesteps. Return = -110.95990485962744
Episode 57 finished after 105 timesteps. Return = -166.47410984245352
Episode 58 finished after 97 timesteps. Return = -156.73295510125288
Episode 59 finished after 125 timesteps. Return = -374.63088816841736
Episode 60 finished after 102 timesteps. Return = -104.08481077087355
Episode 61 finished after 77 timesteps. Return = -215.91439727583327
Episode 62 finished after 97 timesteps. Return = -186.6302828059158
Episode 63 finished after 93 timesteps. Return = -122.6390440332899
Episode 64 finished after 116 timesteps. Return = -83.1758552525774
Episode 65 finished after 108 tim

Episode 170 finished after 102 timesteps. Return = -284.4523883914275
Episode 171 finished after 117 timesteps. Return = -220.1366820679408
Episode 172 finished after 58 timesteps. Return = -186.979505323478
Episode 173 finished after 123 timesteps. Return = -484.90416413501924
Episode 174 finished after 134 timesteps. Return = -94.8135816933842
Episode 175 finished after 65 timesteps. Return = -119.26368398829123
Episode 176 finished after 74 timesteps. Return = -231.0219117660467
Episode 177 finished after 70 timesteps. Return = -182.83065146736593
Episode 178 finished after 104 timesteps. Return = -112.85654844140322
Episode 179 finished after 81 timesteps. Return = -113.5674522833551
Episode 180 finished after 76 timesteps. Return = -307.01901907722265
Episode 181 finished after 95 timesteps. Return = -164.99269739893998
Episode 182 finished after 116 timesteps. Return = -289.68585594369375
Episode 183 finished after 87 timesteps. Return = -334.6190878335255
Episode 184 finished af

Episode 291 finished after 91 timesteps. Return = -96.51374630685024
Episode 292 finished after 62 timesteps. Return = -94.54622098899897
Episode 293 finished after 69 timesteps. Return = -119.57940140864392
Episode 294 finished after 97 timesteps. Return = -290.01738334475
Episode 295 finished after 71 timesteps. Return = -115.15766398232832
Episode 296 finished after 81 timesteps. Return = -319.648145216712
Episode 297 finished after 99 timesteps. Return = -275.8158543977743
Episode 298 finished after 120 timesteps. Return = -106.11979481851704
Episode 299 finished after 86 timesteps. Return = -141.4648956851363
Episode 300 finished after 92 timesteps. Return = -177.81689133774097
Episode 301 finished after 123 timesteps. Return = -47.84569632480145
Episode 302 finished after 101 timesteps. Return = -448.66426100143093
Episode 303 finished after 105 timesteps. Return = -231.55502098310822
Episode 304 finished after 75 timesteps. Return = -131.92576576073836
Episode 305 finished after

Episode 411 finished after 90 timesteps. Return = -98.88539265925864
Episode 412 finished after 72 timesteps. Return = -320.4238494472846
Episode 413 finished after 68 timesteps. Return = -64.64364215083958
Episode 414 finished after 95 timesteps. Return = -308.6433479765385
Episode 415 finished after 68 timesteps. Return = -187.68119998959068
Episode 416 finished after 72 timesteps. Return = -106.45896196934116
Episode 417 finished after 75 timesteps. Return = -308.840500107999
Episode 418 finished after 68 timesteps. Return = -32.65245322663165
Episode 419 finished after 106 timesteps. Return = -247.75608677087263
Episode 420 finished after 79 timesteps. Return = -348.5519095235118
Episode 421 finished after 116 timesteps. Return = -116.47448039321512
Episode 422 finished after 97 timesteps. Return = -239.71960783598783
Episode 423 finished after 108 timesteps. Return = -201.62276161551995
Episode 424 finished after 100 timesteps. Return = -230.02570657887222
Episode 425 finished aft

Episode 531 finished after 107 timesteps. Return = -97.8130140539487
Episode 532 finished after 110 timesteps. Return = -113.57216581831835
Episode 533 finished after 74 timesteps. Return = -89.54221380240719
Episode 534 finished after 64 timesteps. Return = -249.3533325717778
Episode 535 finished after 129 timesteps. Return = -474.5760940135383
Episode 536 finished after 131 timesteps. Return = -184.84173486622774
Episode 537 finished after 117 timesteps. Return = -168.9296341278432
Episode 538 finished after 68 timesteps. Return = -180.42104042873754
Episode 539 finished after 79 timesteps. Return = -123.42821567763775
Episode 540 finished after 98 timesteps. Return = -121.16247434782917
Episode 541 finished after 65 timesteps. Return = -110.64232009487272
Episode 542 finished after 83 timesteps. Return = -122.15477485759845
Episode 543 finished after 117 timesteps. Return = -146.28058867364254
Episode 544 finished after 103 timesteps. Return = -152.84937556574002
Episode 545 finishe

...

Episode 17238 finished after 1000 timesteps. Return = 46.76929699285987
Episode 17239 finished after 220 timesteps. Return = -5.816935088383403
Episode 17240 finished after 146 timesteps. Return = 21.26800343413106
Episode 17241 finished after 257 timesteps. Return = 38.559742369204855
Episode 17242 finished after 188 timesteps. Return = -29.390322660038734
Episode 17243 finished after 200 timesteps. Return = -6.68568647509079
Episode 17244 finished after 1000 timesteps. Return = 81.26077077719191
Episode 17245 finished after 99 timesteps. Return = -13.582420946059855
Episode 17246 finished after 245 timesteps. Return = -176.96483412299716
Episode 17247 finished after 98 timesteps. Return = -182.59557169024606
Episode 17248 finished after 231 timesteps. Return = -76.47022433074252
Episode 17249 finished after 178 timesteps. Return = 54.70078752430197
Episode 17250 finished after 1000 timesteps. Return = 42.115062859972525
Episode 17251 finished after 214 timesteps. Return = -47.3375828

Episode 17352 finished after 429 timesteps. Return = -237.60949459110623
Episode 17353 finished after 1000 timesteps. Return = -0.5526227251524722
Episode 17354 finished after 195 timesteps. Return = -37.283172321569296
Episode 17355 finished after 261 timesteps. Return = -40.89153894645801
Episode 17356 finished after 1000 timesteps. Return = 73.26419398166986
Episode 17357 finished after 1000 timesteps. Return = 48.49925684759008
Episode 17358 finished after 1000 timesteps. Return = 66.39651003328716
Episode 17359 finished after 1000 timesteps. Return = 66.90946976237903
Episode 17360 finished after 1000 timesteps. Return = -10.963668893435129
Episode 17361 finished after 459 timesteps. Return = -253.75716612472863
Episode 17362 finished after 234 timesteps. Return = -64.59450439778226
Episode 17363 finished after 880 timesteps. Return = -406.95817104744214
Episode 17364 finished after 370 timesteps. Return = -196.74642707937363
Episode 17365 finished after 1000 timesteps. Return = 6

Episode 17466 finished after 1000 timesteps. Return = 81.25164361456387
Episode 17467 finished after 1000 timesteps. Return = 34.54635563429803
Episode 17468 finished after 258 timesteps. Return = -15.983726675290143
Episode 17469 finished after 271 timesteps. Return = 25.603836994260448
Episode 17470 finished after 127 timesteps. Return = -194.75019407550144
Episode 17471 finished after 702 timesteps. Return = -182.66218695591795
Episode 17472 finished after 240 timesteps. Return = 18.31108145734929
Episode 17473 finished after 1000 timesteps. Return = 25.95850552525931
Episode 17474 finished after 87 timesteps. Return = -19.05381440810261
Episode 17475 finished after 907 timesteps. Return = -388.42911309051084
Episode 17476 finished after 323 timesteps. Return = -225.04347426143423
Episode 17477 finished after 1000 timesteps. Return = -13.411568025462639
Episode 17478 finished after 1000 timesteps. Return = 85.268462544637
Episode 17479 finished after 121 timesteps. Return = -7.97666

Episode 17580 finished after 149 timesteps. Return = -94.86104688854249
Episode 17581 finished after 191 timesteps. Return = 22.558784666798275
Episode 17582 finished after 186 timesteps. Return = 38.57139655981965
Episode 17583 finished after 223 timesteps. Return = -253.373327997064
Episode 17584 finished after 647 timesteps. Return = -159.31725856885308
Episode 17585 finished after 169 timesteps. Return = -37.006719099872974
Episode 17586 finished after 1000 timesteps. Return = 60.41182313395224
Episode 17587 finished after 1000 timesteps. Return = 59.81594427578259
Episode 17588 finished after 1000 timesteps. Return = 125.75275231420055
Episode 17589 finished after 156 timesteps. Return = -36.432203035191215
Episode 17590 finished after 269 timesteps. Return = -45.89607742407945
Episode 17591 finished after 190 timesteps. Return = -64.18033836591997
Episode 17592 finished after 173 timesteps. Return = -148.3467137013834
Episode 17593 finished after 212 timesteps. Return = -145.9985

KeyboardInterrupt: 

In [165]:
#torch.save(agent.state_dict(), 'checkpoints/lunar_lander_64x64_checkpoint_0.pt')
#agent.load_state_dict(torch.load('checkpoints/lunar_lander_32x32_checkpoint_0.pt'))

In [179]:
visualise_agent(greedy_policy, command=[150, 400], n=5)

Episode 0 finished after 833 timesteps. Return = -360.375805006002


In [181]:
visualise_agent(stochastic_policy, command=[150, 400], n=5)

Episode 0 finished after 213 timesteps. Return = -27.11786348432311
Episode 1 finished after 271 timesteps. Return = -226.95918413035955
Episode 2 finished after 325 timesteps. Return = -26.1805768711521


In [130]:
print([mem['return'] for mem in replay_buffer])

# Previous Code

In [ ]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64):
    all_costs = []
    for i in range(n_updates):
        batch_input = np.zeros((batch_size, np.prod(env.observation_space.shape)+2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_horizon = np.random.randint(1, len(replay_buffer[sample_episode]['observation'])+1)
            sample_mem_idx = np.random.randint(0, len(replay_buffer[sample_episode]['observation'])+1-sample_horizon)
            sample_mem = replay_buffer[sample_episode]['observation'][sample_mem_idx]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_mem_idx]
            batch_input[b] = network_input
            batch_label[b] = label
        batch_input = torch.tensor(batch_input).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_input)
        cost = F.cross_entropy(pred, batch_label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)

In [8]:
def train_net(policy_net, episode_mem, n_samples = 5): #stochastic gradient descent
    all_costs = []
    for i in range(n_samples):
        sample_horizon = np.random.randint(1, len(episode_mem['observation'])+1)
        sample_mem_idx = np.random.randint(0, len(episode_mem['observation'])+1-sample_horizon)
        sample_mem = episode_mem['observation'][sample_mem_idx]
        sample_desired_reward = sum(episode_mem['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
        network_input = torch.tensor(np.append(sample_mem, [sample_desired_reward, sample_horizon])).double()
        label = torch.tensor([episode_mem['action'][sample_mem_idx]]).double()
        
        pred = policy_net(network_input)
        cost = F.binary_cross_entropy(pred, label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)
    

In [33]:
def train(policy_net, n_episodes=100):
    global i_episode
    global epsilon
    try:
        for _ in range(n_episodes):
            observation = env.reset()
            episode_mem = {'observation':[],
                            'action':[],
                            'reward':[],
                            'done':[]}
            done=False
            while not done:
                network_input = torch.tensor(np.append(observation, [desired_reward, command_horizon])).double()
                action_prob = policy_net(network_input)
                action = np.random.binomial(1, action_prob.item())
                #action = int(action_prob.item()>0.5)
                if np.random.rand()<epsilon: action = np.random.randint(0, 2)
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                episode_mem['done'].append(done)
                
                observation=new_observation
                epsilon*=0.999
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            mean_cost = train_net(policy_net, episode_mem)
            
            i_episode+=1
            print("Episode {} finished after {} timesteps. Epsilon={} Mean Cost={}".format(i_episode, len(episode_mem['observation']), epsilon, mean_cost))
        env.close()
    except KeyboardInterrupt:
        env.close()